# CiP Tagging Exercise

In this exercise you will try your hand at assembling an NLP pipeline suggesting tags for [Citypolarna](https://www.citypolarna.se) events.

You will be working with a dataset consisting of descriptions of past events which has already been tagged previously, using them to train a classifier to suggest tags for new and unseen events.

Note that an event may have more than one tag. For example, it may both have **mat** (Swedish for "food") and **konsert** ("concert") for an event where you grab a bite before going to a concert.

## Setup

The first step is to import the packages and data we need for the exercise.

> **Run the cells below to install the necessary packages and download the dataset.**

In [ ]:
# UNCOMMENT THESE BEFORE EXPORTING TO COLAB
#!pip install git+https://github.com/facebookresearch/fastText.git 
#!pip install 'git+https:x//github.com/chrka/cip-tagging-exercise.git#egg=tagger&subdirectory=source'

In [ ]:
from tagger import *

DATASET_URL = "https://cip-tagging-exercise.ngrok.io/"

In [ ]:
from tagger.dataset.external import load_external

events_train, tags_train, events_test, top_tags, tags_train_stats = load_external(
    "https://cip-tagging-exercise.ngrok.io/"
)

## A brief overview of the data

> Feel free to skip through this section and refer back to it only when if you have questions about the format of the data.

The data for the events are contained in the data frames `events_train` (for training your tagger) and `events_test` (events with unkown tags used in the final evaluation). 

The structure of the data frames is as follows:

| Field       | Description                              |  
| ----------- | ---------------------------------------- |  
| id          | ID of the event                          |  
| weekday     | Day of the week (0 = Monday, 6 = Sunday) |  
| time        | Time of day (or _NA_ if an all-day event |  
| title       | Event title                              |  
| description | Event description (HTML)                 |  

The text fields — `title` and `description` — are probably the most important for figuring out which tags should be suggested, and the ones we will be focusing on in this exercise.  But you might like to think about how the other fields could prove to be useful as well.

This is what the data for the first couple of events look like:

In [ ]:
events_train.head()

And we have training data for about 7000 events and will be testing on about 2300 events:

In [ ]:
events_train.info()

In [ ]:
events_test.info()

The classifier will be trained to suggest labels from the following list of tags (shown with their respective counts in the training dataset):

In [ ]:
tags_train_stats

The tags for the events in the training data are available in `tags_train` in matrix form with a row for each event, and where a `1` in the $n$-th column means that the $n$-th tag (in the order given by `top_tags`) was applied for that event.

This is what it looks for the first 5 events:

In [ ]:
tags_train[0:5]

In [ ]:
top_tags

> **TODO**: Add matrix_to_tags to make this simpler

In [ ]:
import numpy as np
np.array(top_tags)[(tags_train[0:1] > 0).squeeze()]

## Pipeline

Now we get to the part where you actually get to put together uour classifier.

The classifier consists of three different parts:

* First, the **preprocessor** which extracts text data from the event data and turns it into tokens, to be used in the next step:
* the **feature extractor** which in turn converts the tokens into numerical data, suitable for use in
* a **Machine Learning algorithm** which learns what tags might be suitable for which events.

The pipeline is built up out of a sequence of **transformers**, each of which does something to the data before passing it onto the next.  At the end of the pipeline, we put our **ML algorithm**.

We define the pipeline as a list of tuples (each being a **name** and the **transformer**/**algorithm** for the step.)

To have something to compare your classifier against, we have also provided a baseline classifier.

> **NB.** Take care that the output of each step matches the input of the next step!

### Preprocessing

The first task of the pipeline is to extract the text from the event data, and turn it into a series of tokens.

The following transformers are suitable for this:

`ExtractText(columns=['description'])`: (_Data frame to HTML_) Extract text fields from event data joined together. By default it only takes the descriptions, but by specifying the `columns` argument you can add other fields as well, eg., `columns=['title', 'description']` to join the titles and descriptions toghether.

`HTMLToText()`: (_HTML to string_) The descriptions are HTML formatted, so we need a way to convert them into raw text without any formatting data.

`CharacterSet(punctuation=True, digits=False)`: (_String to string_) Keeps alphabetic characters (Swedish) and collapses multiple whitespaces into single.  Optionally keeps digits and punctuation.  (Digits have been removed from this particular dataset already however.)

`Lowercase()`: (_String to string_) Converts all alphabetic characters into their lowercase equivalents.

`Tokenize(method='word_punct)`: (_String to token list_) Splits strings into lists of tokens.  If method is `whitespace`, whitespaces are used for splitting, if `word_punct` (default), punctuation marks are also used for splitting.



`Stopwords()`: (_Token list to token list_) Removes stop words (the most common words in the Swedish language).

`Stemming()`: (_Token list to token list_) Converts tokens into their stems.

`NGram(n_min, n_max=None)`: (_Token list to token list_) Create all $n$-grams from $n_{\mathrm{min}}$-grams to $n_{\mathrm{max}}$-grams. (If no $n_{\mathrm{max}}$, only $n_{\mathrm{min}}$-grams are created.)

> Of these steps, `ExtractToText()`, `HTMLToText()`, and, `Tokenize()` are most likely necessary to include in the pipeline, but do try to experiment a little with the other ones as well.

We'll assemble each step of the classifier into a separate [scikit-learn](https://scikit-learn.org/) pipeline so that we can try them out separately if we want to.

> The details of Pipelines are not terribly important right now, but it might be useful to know that we can `fit` them to data, and in the case of transformers, we can use them to `transform` data (after they've been fitted) — classifiers can be used to `predict` (once again, must be fitted first).  As a shortcut, one can also `fit_transform` to fit and transform the same data in one single step.

The baseline model extracts text, converts it from HTML into raw text, removes any non-alphabetic characters — even removing punctuation — and breaks the text into tokens after converting everything into lowercase.

In [ ]:
baseline_preprocessing = Pipeline([
    ('fields', ExtractText()),
    ('html', HTMLToText()),
    ('cset', CharacterSet(punctuation=False)),
    ('lower', Lowercase()),
    ('token', Tokenize())
])

We can take a quick peek at what the baseline's preprocessing steps do (and we do see that it does what we'd expect):

In [ ]:
baseline_preprocessed = baseline_preprocessing.fit_transform(events_train.head())
baseline_preprocessed

> **Now it's your turn to add your own preprocessing steps below.  We have already added the ones that are required.**

In [ ]:
my_preprocessing = Pipeline([
    ('fields', ExtractText()),
    ('html', HTMLToText()),
    # YOUR STEPS HERE
    ('token', Tokenize())
    # YOUR STEPS HERE
])

### Feature Extraction

In order to convert the tokens into numerical data suitable for a machine learning algorithm, you could try one of the following common methods.

`BagOfWords(binary=False)`: (_List of tokens to sparse vector_) Counts the occurences of each word in the list of the tokens, and creates a vector out of them.  If the argument `binary` is set to `True`, then it only cares if a token occurs or not (ie., it gives it a count of either 1 or 0).

> That the result is a _sparse vector_ means that the result only specifies non-zero entries.  When zeros are included as well, it is called a _dense vector_.  Since some implementations of ML algorithms do not work well with sparse vector, we have also provided a `SparseToDense()` transformer below.

`Tfidf()`: (_List of tokens to sparse vector_) Similarly to `BagOfWords`, it also counts the occurences of each token, but instead creates a vector of each token's _term frequency_ (how often the token occurs in the event description) multiplied by its _inverse document frequency_ (one divided by the number of descriptions the token occurs in).  The intuition being that the more often a token occurs in a description the more likely it is that it is representative of that event, while at the same time considering that if the token occurs in many, many events, it is probably not specific to the the event.

`SumWordBedding()`, `MeanWordBedding()`: (_List of tokens to sparse vector_) A different way of converting words to vectors is to use what is known as a _word embedding_. Each word is converted in such a way that words that occur in a similar context, result in vectors that are near each other.  The simplest way of using these are by adding the vectors together for all words (or taking their means).

> **NB.** We have precomputed word vectors from the event dataset based on regular words only (no punctuation, lowercase only) so will probably not work very well with n-grams etc.

Some other transformers that can be useful for dealing with feature vectors:

`SparseToDense()`: (_Sparse vector to dense vector_) Converts a sparse vector to a dense vector with the same contents.  Necessary, for example, for classifiers using the `MultiLayerPerceptron()`.

`MaxAbsScaler()`: (_Sparse/dense vector to sparse/dense vector_) Scales the elements of a vector to be in the range $[-1, 1]$ such that the absolute maximum value of each column (over all training samples) is 1.  This can increase the performance of certain ML algorithms, such as `LogisticRegression()`.

The baseline model uses bag of words, plain and simple.

> If you have experience with [scikit-learn](https://scikit-learn.org/), you may want to try out some other ways of manipulating your features.

In [ ]:
baseline_feature_extraction = Pipeline([
    ('bow', BagOfWords())
])

We can once again take a quick look at what the baseline does:

In [ ]:
baseline_feature_extraction.fit_transform(baseline_preprocessed)

And we see that it results in a sparse matrix for a couple of hundred tokens (since we only look at the first couple of events for this example, the number of distinct tokens won't be that large).

> **Once again, it's your turn! Define your feature extraction steps here:**

In [ ]:
my_feature_extraction = Pipeline([
    # YOUR STEPS HERE
])

### Algorithm

Plenty of algorithms could be used, but here are a couple of suggestions:

`NaiveBayes()`: ((Sparse) Vector to prediction) Naïve Bayes

`LogisticRegression()`: ((Sparse) Vector to predictions) Logistic regression

`MultiLayerPerceptron(layers, epochs=16, batch_size=64)`: (Vector to prediction) Multi-layered perceptron with specified layers, eg., `layers=[256, 256]`)

>  There are many other algorithms in [scikit-learn](https://scikit-learn.org/) you can try if you have some experience.  (Since this is a multi-label problem, you might find [OneVsRestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) useful.)

The baseline uses Naïve Bayes.

In [ ]:
baseline_algorithm = Pipeline([
    ('nb', NaiveBayes())
])

> **Pick your algorithm and define the final step of your classification pipeline below.**

In [ ]:
my_algorithm = Pipeline([
    # YOUR STEPS HERE
])

## Assembling the pipeline

It's now time to put together preprocessing, feature extraction, and the algorithm into a single pipeline.

> **Run the cells below to complete the pipeline for the baseline classifier, and your own.**

In [ ]:
baseline_classifier = Pipeline([
    ('pre', baseline_preprocessing), 
    ('feat', baseline_feature_extraction), 
    ('algo', baseline_algorithm)
])

In [ ]:
my_classifier =  Pipeline([
    ('pre', my_preprocessing), 
    ('feat', my_feature_extraction), 
    ('algo', my_algorithm)
])

## Evaluation

You will now try out your model by training it on a subset of the training data only, and evaluating its performance on the remainder of the data.  This should give you some idea of how well it will perform on unseen data.

The two main metrics the model will be evaluated on are:

* **Hamming loss**: The fraction of tags that either are suggested when they shouldn't be, or aren't suggested when they should. (It ranges from 0–1, lower is better.)
* **Exact match ratio**: The fraction of events that have been completely correctly classified, that is, precisely those labels that should be suggested for the event has been suggested, and no others. (Also ranges from 0–1, but here higher is better.)

It can also be interesting to take a look at how well the classifier works with individual labels, so **accuracy** (how often a predicted tag matches the actual tag), **precision** (how often a suggested tag is correct), **recall** (how often an actual tag is predicted), and **$F_1$-scores** (harmonic mean of precision and recall) are reported for the individual tags as well. (Since any given tag occurs pretty rarely, accuracy is not that important since always predicting that tags shouldn't occur is correct much more often than not.)

> For the sake of simplicity, the classifier only predicts whether or not a tag should be added to an event.  A more realistic case would be to instead predict the _probability_ of a tag being appropriate for an event.

> **Run the cells below to evaluate the performances of the baseline classifier as well as your own.  How do they compare?**

In [ ]:
%%time
evaluate_classifier(baseline_classifier, top_tags, events_train, tags_train)

In [ ]:
%%time
evaluate_classifier(my_classifier, top_tags, events_train, tags_train)

## Submission

Once you have found a sequence of steps you think performs well, it's time to train it on the full training set, and submit its prediction on the (secret!) test set.

> **When you're satisifed with the performance of your classifier, first edit the cell below to add your team name and a brief description of your model, and then run it to see how well it performed compared to the other participants'.**

In [ ]:
%%time
my_classifier.fit(events_train, tags_train)

SUBMISSION_URL = "https://cip-tagging-exercise.ngrok.io/"

submit_model(my_classifier,
             team_name="<INSERT TEAM NAME HERE>",
             model_name="<INSERT MODEL DESCRIPTION HERE>",
             events=events_test,
             base_url=SUBMISSION_URL)